In [ ]:
import pyclesperanto as cle
import numpy as np
import tifffile as tiff
import matplotlib.pyplot as plt
from skimage.io import imread

# Image Processing using clEsperanto

We know how to push and pull data towards our device, we can know apply processing function to it.

The clEsperanto API follow a pattern:

> output = cle.operation_name(input, output, args)

Each functions are documented, you can access though your IDE or using the `cle.operation_name?` idiom un notebook

In [18]:
from skimage.filters import gaussian, threshold_otsu
from skimage.measure import label, regionprops

image = imread("https://imagej.net/ij/images/blobs.gif").squeeze()
blurred_image = gaussian(image, sigma=1)
binary_image = blurred_image > threshold_otsu(blurred_image)
labeled_image = label(binary_image)

fig, axs = plt.subplots(1,4,figsize=(12, 6))
axs[0].imshow(image, cmap='gray')
axs[1].imshow(blurred_image, cmap='gray')
axs[2].imshow(binary_image, cmap='gray')
axs[3].imshow(labeled_image, cmap='nipy_spectral')
for ax in axs:
    ax.axis('off')
plt.show()

/tmp/ipykernel_81867/343848049.py:6: UserWarning: threshold_otsu is expected to work correctly only for grayscale images; image shape (254, 256, 3) looks like that of an RGB image.
  binary_image = blurred_image > threshold_otsu(blurred_image)


ValueError: arrays must be of dtype byte, short, float32 or float64

<Figure size 1200x600 with 4 Axes>